In [1]:
import json
import httpx
import string
from fuzzywuzzy import fuzz
from num2words import num2words
from googletrans import Translator
from tqdm.autonotebook import tqdm

/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [2]:
timeout = httpx.Timeout(30) 
gg_translator = Translator(timeout=timeout)
vi_num2words_dict = {
    0: 'không',
    1: 'một',
    2: 'hai',
    3: 'ba',
    4: 'bốn',
    5: 'năm',
    6: 'sáu',
    7: 'bảy',
    8: 'tám',
    9: 'chín',
    10: 'mười'
}
vi_words2num_dict = {
    'không': 0,
    'một': 1,
    'hai': 2,
    'ba': 3,
    'bốn': 4,
    'năm': 5,
    'sáu': 6,
    'bảy': 7,
    'tám': 8,
    'chín': 9,
    'mười': 10
}

def remove_punc(text):
    text = text.lower()
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)


def get_samples_by_id(image_id, annotations, question_type):
    return_samples = []
    for anno in annotations:
        if anno['image_id'] == image_id:
            if anno['question_type'] == question_type:
                return_samples.append(anno)

    return return_samples

In [4]:
group_questions = []

with open('./data/private-test/evjvqa_private_test-desc-lang-qtype.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
    
test_annotations = test_data['annotations']

In [5]:
group_qids = {}
return_answers = {}

In [6]:
def get_by_language(annotations, language='en'):
    tmp = []
    for anno in annotations:
        if anno['language'] == language:
            tmp.append(anno)

    return tmp

In [7]:
group_qids = []
images_id_processed = []
ignore = []

for i in tqdm(range(len(test_annotations))):
    anno = test_annotations[i]
    image_id = anno['image_id']
    
    question_type = 'HOW_MANY'
    
    if not image_id in images_id_processed:
        if anno['question_type'] == question_type:
            gr_samples = get_samples_by_id(image_id, test_annotations, question_type)

            en = get_by_language(gr_samples, language='en')
            vi = get_by_language(gr_samples, language='vi')
            ja = get_by_language(gr_samples, language='ja')

            max_leng = max(len(en), len(vi), len(ja))
            for j in range(max_leng):
                group = []
                try:
                    group.append(en[j]['id'])
                except:
                    group.append(None)

                try:
                    group.append(vi[j]['id'])
                except:
                    group.append(None)

                try:
                    group.append(ja[j]['id'])
                except:
                    group.append(None)

                group_qids.append(group)
            
            images_id_processed.append(image_id)

len(group_qids)

  0%|          | 0/10000 [00:00<?, ?it/s]

554

In [9]:
with open('./data/private-test/group-howmnay-private.md', 'w', encoding='utf-8') as f:
    for gr in group_qids:
        f.writelines(f"{gr}\n")

In [33]:
group_question_ids = []

for gr in group_qids:
    if not gr in group_question_ids:
        group_question_ids.append(gr)
        
len(group_question_ids)

55

In [31]:
group_question_ids

[[0, 4, 6],
 [1, 5, 7],
 [0, 6, 9],
 [1, 7, 10],
 [2, 8, 11],
 [0, 2, 3],
 [0, 10, 15],
 [1, 11, 16],
 [2, 12, 17],
 [3, 13, 18],
 [4, 14, 19],
 [0, 8, 12],
 [1, 9, 13],
 [2, 10, 14],
 [3, 11, 15],
 [0, 12, 18],
 [1, 13, 19],
 [2, 14, 20],
 [3, 15, 21],
 [4, 16, 22],
 [5, 17, 23],
 [0, 16, 24],
 [1, 17, 25],
 [2, 18, 26],
 [3, 19, 27],
 [4, 20, 28],
 [5, 21, 29],
 [6, 22, 30],
 [7, 23, 31],
 [0, 18, 27],
 [1, 19, 28],
 [2, 20, 29],
 [3, 21, 30],
 [4, 22, 31],
 [5, 23, 32],
 [6, 24, 33],
 [7, 25, 34],
 [8, 26, 35],
 [0, 14, 21],
 [1, 15, 22],
 [2, 16, 23],
 [3, 17, 24],
 [4, 18, 25],
 [5, 19, 26],
 [6, 20, 27],
 [0, 20, 30],
 [1, 21, 31],
 [2, 22, 32],
 [3, 23, 33],
 [4, 24, 34],
 [5, 25, 35],
 [6, 26, 36],
 [7, 27, 37],
 [8, 28, 38],
 [9, 29, 39]]